In [8]:
import pandas as pd

In [9]:
class clean_data:
    
    column_names_first_page = {'Unnamed: 1': 'Sector industrial',
                               'Unnamed: 2': 'Población ocupada por sexo: Hombre',
                               'Unnamed: 3': 'Población ocupada por sexo: Mujer',
                               'Unnamed: 4': 'Ingreso bruto medio en el empleo principal por sexo: Hombre',
                               'Unnamed: 5': 'Ingreso bruto medio en el empleo principal por sexo: Mujer',
                               'Unnamed: 6': 'brecha del ingreso'}
    
    column_names_second_page = {'Unnamed: 1': 'Sector industrial',
                                'Unnamed: 2': 'Población ocupada tiempo parcial por sexo: Hombre',
                                'Unnamed: 3': 'Población ocupada tiempo parcial por sexo: Mujer',
                                'Unnamed: 4': 'Ingreso bruto medio tiempo parcial en el empleo principal por sexo: Hombre',
                                'Unnamed: 5': 'Ingreso bruto medio tiempo parcial en el empleo principal por sexo: Mujer',
                                'Unnamed: 6': 'Diferencial del ingreso tiempo parcial',
                                'Unnamed: 7': 'Población ocupada tiempo completo por sexo: Hombre',
                                'Unnamed: 8': 'Población ocupada tiempo completo por sexo: Mujer',
                                'Unnamed: 9': 'Ingreso bruto medio tiempo completo en el empleo principal por sexo: Hombre',
                                'Unnamed: 10': 'Ingreso bruto medio tiempo completo en el empleo principal por sexo: Mujer',
                                'Unnamed: 11': 'Diferencial del ingreso tiempo completo'}
    
    def clean_first_page(self, df):
        df.drop(df.columns[0], axis=1, inplace=True)
        df.dropna(how='all', inplace=True)
        
        self.data_period = df['Unnamed: 2'][4].split(' ')
        
        df.drop([0,2,4], axis=0, inplace=True)

        df.rename(columns=self.column_names_first_page, inplace=True)
        
        self.total_trabajadores = df.loc[8:8].reset_index(drop=True)
        
        df.drop([5,6,8], axis=0, inplace=True)
        df.reset_index(drop=True, inplace=True)
        
        self.total_sector = df[df['Sector industrial'].str.contains('Sector|1.4 No especificado')].reset_index(drop=True)
        self.sector_publico_privado = df[df['Sector industrial'].str.contains('Público|Privado')].reset_index(drop=True)
        self.Posición_empleo = df[df['Sector industrial'].str.contains('Asalariada|Cuenta propia|Empleadora|Auxiliar no remunerado')].reset_index(drop=True)
        
        df.drop(df[df['Sector industrial'].str.contains('Sector')].index, inplace=True)
        df = df.drop(df.loc[16:39].index).reset_index(drop=True)
        
        if self.data_period[0] == 'I':
            df['Data Period quarterly'] = 1
            df['Data Period year'] = self.data_period[2]
        elif self.data_period[0] == 'II':
            df['Data Period quarterly'] = 2
            df['Data Period year'] = self.data_period[2]
        elif self.data_period[0] == 'III':
            df['Data Period quarterly'] = 3
            df['Data Period year'] = self.data_period[2]
        elif self.data_period[0] == 'IV':
            df['Data Period quarterly'] = 4
            df['Data Period year'] = self.data_period[2]
        
        return df, self.total_trabajadores, self.total_sector, self.sector_publico_privado, self.Posición_empleo
    
    def clean_second_page(self, df):
        df.drop(df.columns[0], axis=1, inplace=True)
        df.dropna(how='all', inplace=True)
        self.data_period = df['Unnamed: 2'][4].split(' ')
        
        self.total_trabajadores = df.loc[10:10].reset_index(drop=True)
        df.drop([0,2,4,5,6,7,8,10], axis=0, inplace=True)

        df.rename(columns=self.column_names_second_page, inplace=True)
        df.reset_index(drop=True, inplace=True)
        
        self.total_sector = df[df['Sector industrial'].str.contains('Sector|1.4 No especificado')].reset_index(drop=True)
        self.sector_publico_privado = df[df['Sector industrial'].str.contains('Público|Privado')].reset_index(drop=True)
        self.Posición_empleo = df[df['Sector industrial'].str.contains('Asalariada|Cuenta propia|Empleadora|Auxiliar no remunerado')].reset_index(drop=True)
        
        df.drop(df[df['Sector industrial'].str.contains('Sector')].index, inplace=True)
        df = df.drop(df.loc[14:39].index).reset_index(drop=True)
        
        if self.data_period[0] == 'I':
            df['Data Period quarterly'] = 1
            df['Data Period year'] = self.data_period[2]
        elif self.data_period[0] == 'II':
            df['Data Period quarterly'] = 2
            df['Data Period year'] = self.data_period[2]
        elif self.data_period[0] == 'III':
            df['Data Period quarterly'] = 3
            df['Data Period year'] = self.data_period[2]
        elif self.data_period[0] == 'IV':
            df['Data Period quarterly'] = 4
            df['Data Period year'] = self.data_period[2]
        
        return df, self.total_trabajadores, self.total_sector, self.sector_publico_privado, self.Posición_empleo

In [10]:
df = pd.read_excel('Data/2018/Oferta_de_empleo_Costa_Rica_4_trimestre_2018.xlsx', sheet_name=1)
df.head(15)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,Cuadro 1,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Costa Rica: Compendio del IV trimestre de 2018...,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Características del empleo,IV Trimestre 2018,NaN,NaN,NaN,NaN
5,NaN,NaN,Población ocupada por sexo,NaN,Ingreso bruto medio en el empleo principal por...,NaN,Brecha del ingreso2/
6,NaN,NaN,Hombre,Mujer,Hombre,Mujer,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,1. Rama de actividad,1332961,832362,467947.25194,405666.402888,86.690626
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
clean = clean_data()
first_page, first_page_total_trabajadores, first_page_total_sector, first_page_sector_publico_privado, first_page_Posición_empleo = clean.clean_first_page(df)
first_page

,Sector industrial,Población ocupada por sexo: Hombre,Población ocupada por sexo: Mujer,Ingreso bruto medio en el empleo principal por sexo: Hombre,Ingreso bruto medio en el empleo principal por sexo: Mujer,brecha del ingreso,Data Period quarterly,Data Period year
0,"Agricultura, ganadería y pesca",229512,36759,334316.274224,374773.703244,112.101543,4,2018
1,Industria manufacturera,154018,83006,419304.673749,336096.435425,80.155662,4,2018
2,Construcción,137657,2424,358193.359928,348158.226811,97.198403,4,2018
3,Otros3/,24568,2375,870981.400962,557126.346316,63.965355,4,2018
4,Comercio y reparación,217753,149391,409698.678646,263725.282086,64.370547,4,2018
5,Transporte y almacenamiento,97429,12185,370475.596764,562934.632734,151.94918,4,2018
6,Hoteles y restaurantes,56809,74162,410051.21817,252698.11445,61.625988,4,2018
7,Intermediación financiera y de seguros,27208,18415,875702.2645,787346.105774,89.910251,4,2018
8,Actividades profesionales y administrativas de...,140016,63611,426446.968007,446988.445957,104.816889,4,2018
9,Administración pública,52331,33875,890258.586599,1057642.172422,118.801682,4,2018


In [12]:
df = pd.read_excel('Data/2018/Oferta_de_empleo_Costa_Rica_4_trimestre_2018.xlsx', sheet_name=2)
df.head(20)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,Cuadro 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Costa Rica: Compendio del IV trimestre de 2018...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Características del empleo,IV Trimestre 2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,Horas efectivas en el empleo principal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,Tiempo parcial1/,NaN,NaN,NaN,NaN,Tiempo completo2/,NaN,NaN,NaN,NaN
7,NaN,NaN,Población ocupada,NaN,Ingreso bruto medio en el empleo principal3/,NaN,Diferencial del ingreso4/,Población ocupada,NaN,Ingreso bruto medio en el empleo principal3/,NaN,Diferencial del ingreso4/
8,NaN,NaN,Hombre,Mujer,Hombre,Mujer,NaN,Hombre,Mujer,Hombre,Mujer,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
second_page, second_page_total_trabajadores, second_page_total_sector, second_page_sector_publico_privado, second_page_Posición_empleo = clean.clean_second_page(df)
second_page

,Sector industrial,Población ocupada tiempo parcial por sexo: Hombre,Población ocupada tiempo parcial por sexo: Mujer,Ingreso bruto medio tiempo parcial en el empleo principal por sexo: Hombre,Ingreso bruto medio tiempo parcial en el empleo principal por sexo: Mujer,Diferencial del ingreso tiempo parcial,Población ocupada tiempo completo por sexo: Hombre,Población ocupada tiempo completo por sexo: Mujer,Ingreso bruto medio tiempo completo en el empleo principal por sexo: Hombre,Ingreso bruto medio tiempo completo en el empleo principal por sexo: Mujer,Diferencial del ingreso tiempo completo,Data Period quarterly,Data Period year
0,"Agricultura, ganadería y pesca",77764,18104,205479.236973,287667.042152,-39.998107,151748,18655,362275.875898,486398.870359,-34.262009,4,2018
1,Industria manufacturera,18973,29209,257237.673731,120207.344505,53.26993,135045,53797,453237.953382,472972.594833,-4.354146,4,2018
2,Construcción,20751,408,215243.297192,66000,69.337024,116906,2016,367414.80802,331776.907001,9.699637,4,2018
3,Otros5/,780,215,538515.36643,199031.007752,63.040793,23788,2160,928551.452673,616505.276792,33.605696,4,2018
4,Comercio y reparación,37648,62154,252401.980311,99995.500671,60.382442,180105,87237,414315.792924,375551.470855,9.356226,4,2018
5,Hoteles y restaurantes,13017,38469,284423.324433,133214.134769,53.163428,43792,35693,379585.421347,368161.883111,3.009478,4,2018
6,Actividades profesionales y administrativas de...,29464,15927,313992.937928,289146.178153,7.913159,110552,47684,468245.446236,498302.647188,-6.419112,4,2018
7,Enseñanza y salud,14494,40936,499759.649097,541750.368911,-8.402183,58747,90294,1104762.771605,881016.852414,20.252847,4,2018
8,Comunicación y otros servicios6/,29080,41009,142573.576692,99869.191434,29.952524,63373,36571,668675.428912,556204.883994,16.819901,4,2018
9,Hogares como empleadores,9652,87774,133939.016865,100328.606791,25.093816,9667,52265,288292.533014,215611.131148,25.21099,4,2018
